In [5]:
import pandas as pd
import numpy as np

import os
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'

import sys

import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display


from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, learning_curve, validation_curve
from sklearn.neighbors import KNeighborsClassifier as kNN
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier


from IPython.display import Audio

from utils.audio_preprocess import *
from utils.septr import SeparableTr
from utils.feature_exctraction_v2 import get3d_data

In [6]:
from tqdm import tqdm

In [7]:
import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Activation, Dense, Conv1D, Conv2D, MaxPooling1D,MaxPooling2D, Flatten, Dropout, BatchNormalization, Input
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

#import warnings
#if not sys.warnoptions:
#    warnings.simplefilter("ignore")
#warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [8]:
# # ave maria 🙏🏻🙏🏿
from scikeras.wrappers import KerasClassifier

In [9]:
Crema = "CREMA-D/AudioWAV/"
crema_directory_list = os.listdir(Crema)

file_emotion = []
file_path = []

NSIZE = len(crema_directory_list)
SAMPLE_SIZE = min(200000, NSIZE)

In [10]:
for file in crema_directory_list[:SAMPLE_SIZE]:

    file_path.append(Crema + file)

    part=file.split('_')
    if part[2] == 'SAD':
        file_emotion.append('sad')
    elif part[2] == 'ANG':
        file_emotion.append('angry')
    elif part[2] == 'DIS':
        file_emotion.append('disgust')
    elif part[2] == 'FEA':
        file_emotion.append('fear')
    elif part[2] == 'HAP':
        file_emotion.append('happy')
    elif part[2] == 'NEU':
        file_emotion.append('neutral')
    else:
        file_emotion.append('Unknown')
        
# dataframe 
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe Emotions/Path
path_df = pd.DataFrame(file_path, columns=['Path'])
Crema_df = pd.concat([emotion_df, path_df], axis=1)
Crema_df['Emotions'] = pd.Categorical(Crema_df['Emotions'])
Crema_df.head()

,Emotions,Path
0,angry,CREMA-D/AudioWAV/1022_ITS_ANG_XX.wav
1,angry,CREMA-D/AudioWAV/1037_ITS_ANG_XX.wav
2,neutral,CREMA-D/AudioWAV/1060_ITS_NEU_XX.wav
3,neutral,CREMA-D/AudioWAV/1075_ITS_NEU_XX.wav
4,disgust,CREMA-D/AudioWAV/1073_IOM_DIS_XX.wav


In [11]:
path = np.array(Crema_df.Path[Crema_df.Emotions=='fear'])[1]
data, sampling_rate = librosa.load(path)

mel = librosa.feature.melspectrogram(y=data, sr=22050) 
mel.shape

(128, 72)

In [12]:
train, test = train_test_split(Crema_df, random_state=42, shuffle=True)

In [ ]:
x_train, y_train, x_test, y_test = get3d_data(train, test, feats = False)

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
# scaling our data with sklearn's Standard scaler
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# x_test = scaler.transform(x_test)

### CNN

In [14]:
def cnn_model(nfilters1=128, nfilters2=64, nfilters3=32, size_kernel=5, nstrides=1, s_pool1=5, s_pool2=5, s_pool3=5):
    model=Sequential()
    model.add(Conv2D(filters=nfilters1, kernel_size=size_kernel, strides=nstrides, padding='same', activation='relu', input_shape=x_train.shape[1:]))
    model.add(Dropout(0.2))    
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=s_pool1, strides = nstrides, padding = 'same'))

    model.add(Conv2D(filters=nfilters2, kernel_size=size_kernel, strides=nstrides, padding='same', activation='relu'))
    model.add(Dropout(0.15))
    model.add(BatchNormalization())    
    model.add(MaxPooling2D(pool_size=s_pool2, strides = nstrides, padding = 'same'))


    model.add(Conv2D(filters=nfilters3, kernel_size=size_kernel, strides=nstrides, padding='same', activation='relu'))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=s_pool3, strides = nstrides, padding = 'same'))

    #model.add(Conv1D(filters=nfilters4, kernel_size=size_kernel, strides=nstrides, padding='same', activation='relu'))
    #model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Flatten())
    model.add(Dense(units=16, activation='relu'))

    model.add(Dense(units=6, activation='softmax'))
    model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

    #rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.30, verbose=0, patience=2, min_lr=0.0000001)

    #model.fit(x_train, y_train, batch_size=32, epochs=25, validation_data=(x_test, y_test), callbacks=[rlrp])    
    
    #model.summary()

    return model

In [15]:
model = KerasClassifier(
    model=cnn_model, 
    verbose=0, 
    batch_size=32, 
    loss="categorical_crossentropy", 
    nfilters1 = 64, 
    nfilters2 = 64, 
    nfilters3 = 64, 
    nfilters4 = 64, 
    size_kernel = 3,
    nstrides = 1,
    size_pool = 5
    )

In [ ]:
param_grid = {
    'nfilters1': [256, 128, 64, 32],
    'nfilters2': [256, 128, 64, 32],
    'nfilters3': [256, 128, 64, 32],
    'nfilters4': [256, 128, 64, 32],
    'size_kernel': [3, 5],
    'nstrides': [1, 2, 3],
    's_pool1': [3, 5, 8],
    's_pool2': [3, 5, 8],
    's_pool3': [3, 5, 8]
    #'optimizer': ['adam']
}

# Grid
grid = RandomizedSearchCV(estimator=cnn_model, param_distributions=param_grid, cv=3, verbose=3, n_jobs=3, n_iter=1000) #GridSearchCV
grid_result = grid.fit(x_train, y_train)

# Print
print("Best hyperparameter: ", grid_result.best_params_)
print("Best accuracy: ", grid_result.best_score_)

best_model = grid_result.best_estimator_
val_accuracy = best_model.score(x_test, y_test)
print("Validaton accuracy: ", val_accuracy)

In [18]:
x_train.shape

(8683, 128, 299, 1)

In [ ]:
cnn_model().fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=32)

### SepTr